In [1]:
import ROOT
import pandas as pd
import os
from array import array

Welcome to JupyROOT 6.30/02


In [2]:
def getCanvas():

    canvas = ROOT.TCanvas("", "", 1200, 800)

    canvas.Divide(1, 2)

    canvas.cd(1).SetPad(0., 0.3, 1.0, 1.0)
    canvas.cd(1).SetTopMargin(0.15)
    canvas.cd(1).SetBottomMargin(0.05)
    canvas.cd(1).SetRightMargin(0.05)
    canvas.cd(1).SetLeftMargin(0.20)
    # canvas.cd(1).SetLeftMargin(0.15)
    canvas.cd(1).SetTicks(1, 1)


    canvas.cd(2).SetPad(0., 0.0, 1.0, 0.3)
    canvas.cd(2).SetTopMargin(0.1)
    canvas.cd(2).SetBottomMargin(0.2)
    canvas.cd(2).SetRightMargin(0.05)
    canvas.cd(2).SetLeftMargin(0.20)
    canvas.cd(2).SetTicks(1, 1)

    return canvas


In [3]:
def getErrorBand(hist, color = ROOT.kBlack):

    statErrorband = ROOT.TGraphAsymmErrors(hist)


    for i in range(hist.GetNbinsX()):
        statErrorband.SetPointEYlow(i, hist.GetBinError(i+1))
        statErrorband.SetPointEYhigh(i, hist.GetBinError(i+1))
        statErrorband.SetPointEXlow(i, hist.GetBinWidth(i+1)/2.)
        statErrorband.SetPointEXhigh(i, hist.GetBinWidth(i+1)/2.)


    statErrorband.SetFillStyle(3004)
    statErrorband.SetLineColor(color)
    statErrorband.SetFillColor(color)

    return statErrorband

    

In [4]:
def getLegend():

    legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
    # legend.AddEntry(hist, name, "l")

    return legend


In [5]:
def getRatio(hist1, hist2, color):

    nbins = hist1.GetNbinsX()
    
    bin_edges = []
    for i in range(1, nbins + 2):  # Iterate over bins and overflow bin
        bin_edges.append(hist1.GetXaxis().GetBinLowEdge(i))
    bin_edges.append(hist1.GetXaxis().GetBinUpEdge(hist1.GetNbinsX()))  

    bin_edges = array('d', bin_edges)
    # print("bin range ", bin_edges)
    ratio = ROOT.TH1F(
        "", "", nbins, bin_edges)

    # Subtract histogram2 from histogram1 and fill the new histogram
    ratio.Divide(hist1, hist2)
    # ratio.GetXaxis().SetTitle("")
    ratio.GetYaxis().SetTitle("ratio")
    ratio.GetYaxis().SetRangeUser(0.3, 1.7)
    # Modify the font size of the y-axis label
    # Set the font size for the y-axis labels
    ratio.GetYaxis().SetLabelSize(0.13)
    ratio.GetYaxis().SetTitleSize(0.13)
    ratio.GetYaxis().SetTitleOffset(0.5)  # Set

    # Set the font size for the x-axis labels
    ratio.GetXaxis().SetLabelSize(0.13)
    # Modify the font size of the x-axis title
    # Set the font size for the x-axis title
    ratio.GetXaxis().SetTitleSize(0.13)
    ratio.GetXaxis().SetTitleOffset(1.0)  # Set the offset of the x-axis title


    ratio.SetStats(0)
    ratio.GetYaxis().SetNdivisions(505)
    ratio.SetLineColor(color)
    ratio.SetLineWidth(1)
    ratio.SetMarkerStyle(20)

    # ROOT.gStyle.SetErrorX(0)

    # ratio.GetXaxis().SetRangeUser(user_binrange[0], user_binrange[1])

    return ratio
        

In [6]:
def getRatioLine(ratio):

    nbins = ratio.GetNbinsX() 
    # print(nbins)

    bin_edges = []
    for i in range(1, nbins + 2):  # Iterate over bins and overflow bin
        bin_edges.append(ratio.GetXaxis().GetBinLowEdge(i))
    bin_edges.append(ratio.GetXaxis().GetBinUpEdge(ratio.GetNbinsX())) 

    bin_edges = array('d', bin_edges)
    ratio_line = ROOT.TH1F(
        "", "", nbins, bin_edges)

    for i in range(nbins+1):
        ratio_line.SetBinContent(i, 1)
        # ratio_line.SetBinError(i, 1)
    ratio_line.GetYaxis().SetRangeUser(0., 2.0)

    ratio_line.SetLineWidth(1)
    ratio_line.SetLineColor(ROOT.kBlack)

    # ratio_line.GetXaxis().SetRangeUser(user_binrange[0], user_binrange[1])

    return ratio_line


In [7]:
def saveCavas(canvas, name, path):

    canvas.SaveAs(path+"/"+name+".png")


In [8]:
def getHist(file, histofullname, node, event, color):


    hist = file.Get(histofullname) 

    hist.SetStats(False)

    # if hist.Integral() > 0:
    #     hist.Scale(1.0 / hist.Integral())
    # else:
    #     print("histogram {} has 0 integral".format(category))

    hist.SetLineColor(color)
    hist.SetLineWidth(1)
    hist.SetFillColor(0)
    # hist.SetFillStyle(1001)
    

    hist.GetXaxis().SetTitle("final discriminator in "+node+" for event "+event)
    hist.GetYaxis().SetTitle("Number of events")
    # hist.GetXaxis().SetRangeUser(binrange[0], binrange[1])

    # histSetup = SetupPlot(hist, binrange, color=color)
    # hist.SetTitle(event + " event in "+ node+ ' node')
    # print (hist)

    return hist


In [9]:

base_path = "/Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults"
sample = "/Run2-1107/"
# file = "output_limit.root"
csv_file = "/Users/wei/Desktop/SM_TTHH/TTHHRun2/NuisanceParameter/systList-2018.csv"
histoname = 'ljets_ge4j_ge3t_'
events = ['ttHH', 'ttH','ttZ','ttZH','ttZZ','ttbar']
# events = ['ttHH', 'ttH','ttZ','ttZH','ttZZ','ttlf','ttcc','ttmb','ttnb']

root_file = ROOT.TFile(base_path+sample+"/output_limit.root", "READ")
df = pd.read_csv(csv_file)
systs = df['Uncertainty'].astype(str).to_numpy()

color1 = ROOT.kBlack
color2 = ROOT.kRed
color3 = ROOT.kBlue



In [10]:
for node in events:

    if not os.path.exists(base_path+sample+node):
        os.mkdir(base_path+sample+node)

    for event in events:

        hist = getHist(root_file, histoname+node+"_node__"+event, node, event, color1)

        for syst in systs:

            if not os.path.exists(base_path+sample+node+"/"+syst):
                os.mkdir(base_path+sample+node+"/"+syst)
            # print(syst)
            # print(df[df['Uncertainty']==syst][event].values[0])
            if df[df['Uncertainty']==syst][event].values[0] == '1':

                # print(syst)

                histup = getHist(root_file, histoname+node+"_node__"+event+"__"+syst+"Up", node, event, color2)
                histdown = getHist(root_file, histoname+node+"_node__"+event+"__"+syst+"Down", node, event, color3) 

                ErrorBand = getErrorBand(hist, color1)
                ErrorBandup = getErrorBand(histup, color2)
                ErrorBanddown = getErrorBand(histdown, color3)

                legend1 = getLegend()
                legend1.AddEntry(hist, "nominal", "l")
                legend1.AddEntry(histup, syst+" up", "l")
                # legend.AddEntry(histdown, syst+" down", "l")

                legend2 = getLegend()
                legend2.AddEntry(hist, "nominal", "l")
                # legend2.AddEntry(histup, syst+" up", "l")
                legend2.AddEntry(histdown, syst+" down", "l")


                ratioup = getRatio(histup, hist, color2)
                ratiodown = getRatio(histdown, hist, color3)
                ratio_line = getRatioLine(ratioup)


                canvas = getCanvas()
                canvas.cd(1)
                canvas.cd(1).SetLogy()
                # ROOT.gStyle.SetErrorX(1)
                histup.Draw("HIST")
                hist.Draw("HISTSAME")
                # histdown.Draw("HISTSAME")
                
                ErrorBandup.Draw("SAME2")
                ErrorBand.Draw("SAME2")
                # ErrorBanddown.Draw("SAME2")
                legend1.Draw("SAME")


                canvas.cd(2)
                ROOT.gStyle.SetErrorX(0)
                ratioup.Draw()
                # ratiodown.Draw("SAME")
                ratio_line.Draw("SAME")

                canvas.Update()

                canvas.SaveAs(base_path+sample+node+"/"+syst+"/"+node+"_node_"+event+"_event_"+syst+"Up.png")

                canvas.Close()


                canvas = getCanvas()
                canvas.cd(1)
                canvas.cd(1).SetLogy()
                # ROOT.gStyle.SetErrorX(1)
                # histup.Draw("HIST")
                hist.Draw("HIST")
                histdown.Draw("HISTSAME")
                
                # ErrorBandup.Draw("SAME2")
                ErrorBand.Draw("SAME2")
                ErrorBanddown.Draw("SAME2")
                legend2.Draw("SAME")


                canvas.cd(2)
                ROOT.gStyle.SetErrorX(0)
                # ratioup.Draw()
                ratiodown.Draw()
                ratio_line.Draw("SAME")

                canvas.Update()

                canvas.SaveAs(base_path+sample+node+"/"+syst+"/"+node+"_node_"+event+"_event_"+syst+"Down.png")

                canvas.Close()

Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults/Run2-1107/ttHH/CMS_eff_m_trigger_2018/ttHH_node_ttHH_event_CMS_eff_m_trigger_2018Up.png has been created
Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults/Run2-1107/ttHH/CMS_eff_m_trigger_2018/ttHH_node_ttHH_event_CMS_eff_m_trigger_2018Down.png has been created
Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults/Run2-1107/ttHH/CMS_eff_e_trigger_2018/ttHH_node_ttHH_event_CMS_eff_e_trigger_2018Up.png has been created
Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults/Run2-1107/ttHH/CMS_eff_e_trigger_2018/ttHH_node_ttHH_event_CMS_eff_e_trigger_2018Down.png has been created
Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults/Run2-1107/ttHH/CMS_eff_m/ttHH_node_ttHH_event_CMS_eff_mUp.png has been created
Info in <TCanvas::Print>: png file /Users/wei/Desktop/SM_TTHH/TTHHRun2/EvalResults